In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paramètres
img_size = (150, 150)
batch_size = 32

# Création du générateur avec rescaling et une séparation pour la validation (ex: 20%)
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    zoom_range=0.2  # Augmentation par zoom
)

# Création des flux d'images pour l'entraînement et la validation
train_generator = datagen.flow_from_directory(
    directory="C:\Users\bintu\Downloads\Brain tumor classification with simple CNN.zip",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    directory="C:\Users\bintu\Downloads\Brain tumor classification with simple CNN.zip",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


In [ ]:
print("Nombre d'images d'entraînement:", train_generator.samples)
print("Nombre d'images de validation:", val_generator.samples)


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Les labels (classes) dans le générateur sont stockés sous forme d'entiers
classes = train_generator.classes
class_indices = train_generator.class_indices
# Inverser le mapping pour obtenir les noms à partir des indices
inv_class_indices = {v: k for k, v in class_indices.items()}

# Création d'un DataFrame pour la visualisation
df = pd.DataFrame({'Classe': [inv_class_indices[c] for c in classes]})

plt.figure(figsize=(6,4))
sns.countplot(x='Classe', data=df)
plt.title("Répartition des classes dans le jeu d'entraînement")
plt.show()


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.InputLayer(input_shape=(150, 150, 3)),

    # Première couche de convolution
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    # Deuxième couche de convolution
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    # Couche dense 1
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    # Couche dense 2
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    # Couche de sortie
    layers.Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop]
)


In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy entraînement')
plt.plot(history.history['val_accuracy'], label='Accuracy validation')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Evolution de l'Accuracy")

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss entraînement')
plt.plot(history.history['val_loss'], label='Loss validation')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.legend()
plt.title("Evolution de la Loss")
plt.show()


In [ ]:
model.save("brain_tumor_classifier.h5")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Récupérer un batch d'images du jeu de validation
val_images, val_labels = next(val_generator)
predictions = model.predict(val_images)
pred_labels = (predictions > 0.5).astype(int)

# Affichage de 9 images avec leur étiquette réelle et prédite
plt.figure(figsize=(12, 12))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(val_images[i])
    plt.title(f"Réel: {val_labels[i]} / Prédit: {pred_labels[i][0]}")
    plt.axis('off')
plt.show()
